# Equations

$ \delta = [R_{t+1} + \gamma Q(s_{t+1}, a_{t+1})] - Q(s_t,a_t) $

$ Q(s_t,a_t) = Q(s_t,a_t) + \alpha\delta $

## SARSA

In [ ]:
#from plotly.graph_objs import Scatter, Layout
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
def TD(nstates,nepisodes,lrate,gamma,td_lambda): 
    n = 1024
    #nstates = 50
    nactions = 2
    goal = 0
    reward = np.zeros(nstates)
    reward[goal] = 1
    
    states = hrr.hrrs(n,nstates)
    actions = hrr.hrrs(n,nactions)
    
    stateactions = hrr.oconvolve(actions,states)
    stateactions = np.reshape(stateactions,(nstates,nactions,n))
    
    W = hrr.hrr(n)
    bias = 1

    #lrate = 0.1
    eligibility = np.zeros(n)
    #gamma = 0.9
    #td_lambda = 0.5
    epsilon = 0.05
    #nepisodes = 10000
    nsteps = 100
    
    for episode in range(nepisodes):
        state = random.randrange(0,nstates)
        values = np.dot(stateactions[state],W) + bias
        action = np.argmax(values)
        if random.random() < epsilon:
            action = random.randrange(0,nactions)
            
        eligibility = np.zeros(n)
        
        for step in range(nsteps):
            r = reward[state]
            if state == goal:
                eligibility = stateactions[state,action,] + td_lambda*gamma*eligibility
                error = r - values[action]
                W += lrate*error*eligibility
                break
                
            pstate = state
            pvalues = values
            paction = action
            
            eligibility = stateactions[state,action,] + td_lambda*gamma*eligibility
            
            state = ((state+np.array([-1,1]))%nstates)[action]
            
            values = np.dot(stateactions[state],W) + bias 
            action = np.argmax(values)
            if random.random() < epsilon:
                action = random.randrange(0,nactions)
                
            error = (r+gamma*values[action])-pvalues[paction]
            W += lrate*error*eligibility
            
        
    V1 = list(map(lambda x: np.dot(x,W)+bias, stateactions[:,0,:]))
    V2 = list(map(lambda x: np.dot(x,W)+bias, stateactions[:,1,:]))
    
    plotly.offline.iplot([
    dict(x=[x for x in range(len(V1))] , y=V1, type='scatter',name='left'),
    dict(x=[x for x in range(len(V1))] , y=V2, type='scatter',name='right')
    ])
   

In [ ]:
TD(50,10000,.1,.9,.5)
#inputs: nstates,nepisodes,lrate,gamma,td_lambda